In [2]:
%cd learners

[Errno 2] No such file or directory: 'learners'
/home/buehrle/dev/InteractionImitation/scratch/etienne/pillbox/learners


In [ ]:
import gym
from tqdm import tqdm

def rollout(pi, max_steps=1000):
    env = gym.make('intersim:intersim-v0')
    env.reset() # obs = env.reset()
    obs, _, done, _ = env.step(0 * env.action_space.sample())
    
    _except = lambda o, i: torch.cat((o[:i], o[i+1:]))
    
    _relative_state_v = lambda obs: torch.stack((
        obs[..., 0],
        obs[..., 1],
        (obs[..., 2]**2 + obs[..., 3]**2).sqrt(),
        obs[..., 4],
        obs[..., 5],
    ), -1)
    
    for _ in tqdm(range(max_steps)):
        pi_obs = [
            torch.cat((e.unsqueeze(0), _relative_state_v(_except(o, i)))).unsqueeze(0)
            for i, (e, o) in enumerate(zip(obs['state'], obs['relative_state']))
        ]
        
        actions = [pi(o).squeeze() for o in pi_obs]
        actions = torch.stack(actions).unsqueeze(1)
        obs, _, done, _ = env.step(actions)
        env.render(mode='post')
        if done:
            break
    env.close()

In [4]:
import torch
import numpy as np

def expert(obs):
    ego = obs[:, 0]
    rel = obs[:, 1:]
    front = torch.stack((torch.cos(ego[:, 3]), torch.sin(ego[:, 3])), -1)
    left = torch.stack((-torch.sin(ego[:, 3]), torch.cos(ego[:, 3])), -1)
    df = (rel[:, :, :2] * front.unsqueeze(1)).sum(-1)
    dl = (rel[:, :, :2] * left.unsqueeze(1)).sum(-1)

    df = torch.where(df.isnan(), np.inf * torch.ones_like(df), df)
    dl = torch.where(dl.isnan(), np.inf * torch.ones_like(dl), dl)
    rel = torch.where(rel.isnan(), np.inf * torch.ones_like(rel), rel)

    # relative speed in direction of position difference vector
    vrel = rel[:, :, 2] * (rel[:, :, :2] * torch.stack((
        torch.cos(ego[:, 3].unsqueeze(1) + rel[:, :, 3]),
        torch.sin(ego[:, 3].unsqueeze(1) + rel[:, :, 3])),
        -1)).sum(-1)
    vrel = torch.where(vrel.isnan(), np.inf * torch.ones_like(vrel), vrel)
    vrel = torch.maximum(vrel, torch.zeros_like(vrel))
    
    alpha = torch.atan2(dl, df)
    d = (rel[:, :, :2] ** 2).sum(-1)
    attn = torch.exp(-torch.where(alpha > 0, 0.8*alpha, 1*alpha)**2 - 0.01 * d - 0.1*vrel) 
    
    act = 10 - ego[:, 2] - 20 * attn.sum(-1)
    
    return act

In [5]:
rollout(expert, max_steps=500)

Vehicle Trajectory Paths: /home/buehrle/dev/InteractionImitation/InteractionSimulator/datasets/trackfiles/DR_USA_Roundabout_FT/vehicle_tracks_000.csv
Map Path: /home/buehrle/dev/InteractionImitation/InteractionSimulator/datasets/maps/DR_USA_Roundabout_FT.osm
Environment Reset


  0%|          | 0/500 [00:00<?, ?it/s]


RuntimeError: torch.cat(): Sizes of tensors must match except in dimension 0. Got 5 and 6 in dimension 1 (The offending index is 1)